In [ ]:
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer
import shap
import transformers
import pandas as pd
# Create class for data preparation
class SimpleDataset:
    def __init__(self, tokenized_texts):
        self.tokenized_texts = tokenized_texts
    
    def __len__(self):
        return len(self.tokenized_texts["input_ids"])
    
    def __getitem__(self, idx):
        return {k: v[idx] for k, v in self.tokenized_texts.items()}

In [ ]:
fulldata=pd.read_csv('fulldata.csv')

In [ ]:
def find_sus(label):
    if label.find('Sustaining proliferative signaling') == -1:
        return 0
    else: 
        return 1

def find_data(label):
    if label.find('Resisting cell death') != -1:
        return 1
    else: 
        if label.find('Sustaining proliferative signaling') != -1:
            return 1
        else:
            if label.find('Tumor promoting inflammation') != -1:
                return 1
            else:
                return 0


In [ ]:
fulldata['label1']=fulldata['label'].apply(find_sus)

In [ ]:
fulldata['label2']=fulldata['label'].apply(find_data)

In [ ]:
#newdata=fulldata[fulldata['label2']==1]

In [ ]:
dataset=pd.DataFrame(data={'text':[],'label':[]})
dataset['text']=fulldata['text']
dataset['label']=fulldata['label1']

In [ ]:
dataset.to_csv('fulldataset.csv',index=False)

In [ ]:
'''hocdataset=pd.DataFrame(data={'text':[],'label':[]})
hocdataset['text']=newdata['text']
hocdataset['label']=newdata['label1']'''

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(dataset, test_size=0.2)

In [ ]:
''''train.to_csv('train.csv',index=False)
test.to_csv('test.csv',index=False)
np.save('test.npy',test)'''

"'train.to_csv('train.csv',index=False)\ntest.to_csv('test.csv',index=False)\nnp.save('test.npy',test)"

In [ ]:
train.to_csv('fulltrain.csv',index=False)
test.to_csv('fulltest.csv',index=False)
np.save('fulltest.npy',test)

In [ ]:
'''from datasets import load_dataset
raw_datasets=load_dataset('csv',data_files={'train': 'train.csv',
                                              'test':'test.csv'})'''

"from datasets import load_dataset\nraw_datasets=load_dataset('csv',data_files={'train': 'train.csv',\n                                              'test':'test.csv'})"

In [ ]:
from datasets import load_dataset
raw_datasets=load_dataset('csv',data_files={'train': 'fulltrain.csv',
                                              'test':'fulltest.csv'})

Using custom data configuration default-d6675006ef2dae1a


100%|██████████| 2/2 [00:00<00:00, 1002.94it/s]


Dataset csv downloaded and prepared to C:\Users\RmmLeo10\.cache\huggingface\datasets\csv\default-d6675006ef2dae1a\0.0.0\6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e. Subsequent calls will reuse this data.


100%|██████████| 2/2 [00:00<00:00, 125.42it/s]


In [ ]:
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer

# Create class for data preparation
class SimpleDataset:
    def __init__(self, tokenized_texts):
        self.tokenized_texts = tokenized_texts
    
    def __len__(self):
        return len(self.tokenized_texts["input_ids"])
    
    def __getitem__(self, idx):
        return {k: v[idx] for k, v in self.tokenized_texts.items()}

In [ ]:
tokenizer_name = "distilbert-base-uncased"
model_name = "distilbert-base-uncased-finetuned-sst-2-english"

tokenizer = transformers.DistilBertTokenizerFast.from_pretrained(tokenizer_name)
model = transformers.DistilBertForSequenceClassification.from_pretrained(model_name).to('cuda')
trainer = Trainer(model=model)
#pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True,device=0)

loading file https://huggingface.co/distilbert-base-uncased/resolve/main/vocab.txt from cache at C:\Users\RmmLeo10/.cache\huggingface\transformers\0e1bbfda7f63a99bb52e3915dcf10c3c92122b827d92eb2d34ce94ee79ba486c.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/distilbert-base-uncased/resolve/main/tokenizer.json from cache at C:\Users\RmmLeo10/.cache\huggingface\transformers\75abb59d7a06f4f640158a9bfcde005264e59e8d566781ab1415b139d2e4c603.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4
loading file https://huggingface.co/distilbert-base-uncased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/distilbert-base-uncased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/distilbert-base-uncased/resolve/main/tokenizer_config.json from cache at C:\Users\RmmLeo10/.cache\huggingface\transformers\8c8624b8ac8aa99c60c912161f8332de003484428c47906d7ff7eb7f

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

100%|██████████| 1/1 [00:00<00:00,  4.70ba/s]


In [ ]:
full_train_dataset = tokenized_datasets["train"]
full_eval_dataset = tokenized_datasets["test"]

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments("test_trainer")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate = 2e-5,
    #batch_size = 32,
    warmup_steps= 0,
    #max_seq_length = 128,
    num_train_epochs = 5.0 ,
    weight_decay=0.01 
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model, args=training_args, train_dataset=full_train_dataset, eval_dataset=full_eval_dataset
)

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text.
This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
***** Running training *****
  Num examples = 1481
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 930
 54%|█████▍    | 500/930 [00:45<00:39, 10.91it/s]Saving model checkpoint to ./results\checkpoint-500
Configuration saved in ./results\checkpoint-500\config.json


{'loss': 0.2597, 'learning_rate': 9.24731182795699e-06, 'epoch': 2.69}


Model weights saved in ./results\checkpoint-500\pytorch_model.bin
100%|█████████▉| 929/930 [01:26<00:00, 10.66it/s]

Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 930/930 [01:26<00:00, 10.73it/s]

{'train_runtime': 86.701, 'train_samples_per_second': 85.408, 'train_steps_per_second': 10.727, 'train_loss': 0.15510610867572086, 'epoch': 5.0}


TrainOutput(global_step=930, training_loss=0.15510610867572086, metrics={'train_runtime': 86.701, 'train_samples_per_second': 85.408, 'train_steps_per_second': 10.727, 'train_loss': 0.15510610867572086, 'epoch': 5.0})

In [ ]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=full_train_dataset,
    eval_dataset=full_eval_dataset,
    compute_metrics=compute_metrics,
)
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 371
  Batch size = 8
100%|██████████| 47/47 [00:01<00:00, 35.06it/s]


{'eval_loss': 0.6709614396095276,
 'eval_accuracy': 0.889487870619946,
 'eval_runtime': 1.3752,
 'eval_samples_per_second': 269.774,
 'eval_steps_per_second': 34.176}

In [ ]:
trainer.save_model('C:\RMM\Medical1\distilbert-base-uncased-finetuned-sst-2-english2')

Saving model checkpoint to C:\RMM\Medical1\distilbert-base-uncased-finetuned-sst-2-english2
Configuration saved in C:\RMM\Medical1\distilbert-base-uncased-finetuned-sst-2-english2\config.json
Model weights saved in C:\RMM\Medical1\distilbert-base-uncased-finetuned-sst-2-english2\pytorch_model.bin


In [ ]:
from transformers import DistilBertForSequenceClassification, DistilBertConfig, AutoModelForTokenClassification

# retreive the saved model 
model = DistilBertForSequenceClassification.from_pretrained('C:\RMM\Medical1\distilbert-base-uncased-finetuned-sst-2-english2', 
                                                        local_files_only=True)

model.to('cuda')

loading configuration file C:\RMM\Medical1\distilbert-base-uncased-finetuned-sst-2-english2\config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased-finetuned-sst-2-english",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "finetuning_task": "sst-2",
  "hidden_dim": 3072,
  "id2label": {
    "0": "NEGATIVE",
    "1": "POSITIVE"
  },
  "initializer_range": 0.02,
  "label2id": {
    "NEGATIVE": 0,
    "POSITIVE": 1
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.16.2",
  "vocab_size": 30522
}

loading weights file C:\RMM\Medical1\distilbert-base-

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

In [ ]:
from transformers import TextClassificationPipeline
pipe=TextClassificationPipeline(tokenizer=tokenizer,model=model,device=0)

In [ ]:
cor_x = np.load("fulltest.npy",allow_pickle=True)
cor_reviews = [review[0] for review in cor_x]
cor_labels = [review[1] for review in cor_x]

In [ ]:
def score_and_visualize(text):
  prediction = pipe([text])
  print(prediction[0])

  explainer = shap.Explainer(pipe)
  shap_values = explainer([text])

  shap.plots.text(shap_values)

In [ ]:
def mask_top_k(k, pred_label_no_mask, values, returned_tokens):
    """
    masked the k tokens that have the max shap values
    :param k: specify the largest k value
    :param values: shap values
    :param returned_tokens: a list of tokens
    :return: review, which is a str constructed from a list words
    """
    shap_values_neg, shap_values_pos = zip(*values)
    values = shap_values_neg if pred_label_no_mask==0 else shap_values_pos
    # print(values)
    values = np.array(values)
    # ids_top_k = np.argpartition(values, -k)[-k:]
    ids_top_k = (-values).argsort()[:k]
    for idx in ids_top_k:
        # print(idx)
        returned_tokens[idx] = "[UNK] "
    masked_review = "".join(returned_tokens)
    # print(masked_review)
    return masked_review

In [ ]:
def predict_label(pipe, masked_review):
    """
    predict the label for the masked_review
    :param pipe: pipeline
    :param masked_review: string
    :return: 0 or 1, indicating the label
    """
    prediction = pipe([masked_review])
    if prediction[0]['label'] == 'NEGATIVE':
            neg_score = prediction[0]["score"]
            pos_score=1-prediction[0]["score"]
    else:
            pos_score = prediction[0]["score"] 
            neg_score = 1-prediction[0]["score"]       
    pred_label = 0 if prediction[0]['label'] == 'NEGATIVE'  else 1
    return pred_label

In [ ]:
def model_prediction_gpu(x):
    tv = torch.tensor([tokenizer.encode(v, padding='max_length', 
                                        max_length=90, truncation=True) for v in x]).cuda()
    attention_mask = (tv!=0).type(torch.int64).cuda()
    outputs = model(tv, attention_mask=attention_mask)[0]
    scores = torch.nn.Softmax(dim=-1)(outputs)
    val = torch.logit(scores).detach().cpu().numpy()

    return val

In [ ]:
shap_values_list = []
token_data_list = []
top_k = [1, 5, 9, 13]
all_labels =[]
# use GPU
gpu_explainer = shap.Explainer(model_prediction_gpu, tokenizer)
i = 0
for review, label in zip(cor_reviews, cor_labels):
    print(f"process {i}-th review")
    i += 1
    label4review =[]
    label4review.append(label)
    # to-do: truncate review if len(review)>80
    tokens = tokenizer.tokenize(review)
    if len(tokens) > 80:
        tokens_truncated = tokens[:80]
        review = " ".join(token for token in tokens_truncated)
    pred_label_no_mask = predict_label(pipe, review) # predicted label for review without mask
    label4review.append(pred_label_no_mask)
    shap_values = gpu_explainer(review)
    values = shap_values.values[0] # 2-dim ndarray
    returned_tokens = shap_values.data[0]
    for k in top_k:
        masked_review = mask_top_k(k, pred_label_no_mask, values, returned_tokens) # mask review by the shap values
        predicted_label= predict_label(pipe, masked_review)
        label4review.append(predicted_label)
    # label4review = [True_label, pred_label_without_mask, masked_label_1, masked_label_2, masked_review_3, masked_review_4]
    all_labels.append(label4review)
    with open("labels.csv", mode="a") as f:
        f.write("%s\n" % ",".join(str(label) for label in label4review))


process 0-th review
process 1-th review


Token indices sequence length is longer than the specified maximum sequence length for this model (671 > 512). Running this sequence through the model will result in indexing errors


process 2-th review


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


process 3-th review
process 4-th review
process 5-th review
process 6-th review
process 7-th review
process 8-th review
process 9-th review
process 10-th review
process 11-th review
process 12-th review
process 13-th review
process 14-th review
process 15-th review
process 16-th review
process 17-th review
process 18-th review
process 19-th review
process 20-th review
process 21-th review
process 22-th review
process 23-th review
process 24-th review
process 25-th review
process 26-th review
process 27-th review
process 28-th review
process 29-th review
process 30-th review
process 31-th review
process 32-th review
process 33-th review
process 34-th review
process 35-th review
process 36-th review
process 37-th review
process 38-th review
process 39-th review
process 40-th review
process 41-th review
process 42-th review
process 43-th review
process 44-th review
process 45-th review
process 46-th review
process 47-th review
process 48-th review
process 49-th review
process 50-th review

In [ ]:
np.save("all_labels.npy", np.array(all_labels))

In [ ]:
true_label, predicted_label_without_mask, predicted_label_mask1, predicted_label_mask5, predicted_label_mask9, predicted_label_mask13 = zip(*all_labels)

In [ ]:
true_label =[int(ele) for ele in true_label]

In [ ]:
import pandas as pd
df = pd.DataFrame(np.array(all_labels))

In [ ]:
df

,0,1,2,3,4,5
0,1,0,0,0,0,0
1,0,0,0,0,0,0
2,0,0,1,0,0,0
3,0,0,1,0,0,0
4,0,0,0,0,0,0
...,...,...,...,...,...,...
366,1,0,1,0,0,0
367,1,0,0,0,0,0
368,0,0,0,0,0,0
369,0,0,0,0,0,0


In [ ]:
filtered_df = df.loc[df[0]==df[1]]
len(filtered_df)

287

In [ ]:
filtered_df

,0,1,2,3,4,5
1,0,0,0,0,0,0
2,0,0,1,0,0,0
3,0,0,1,0,0,0
4,0,0,0,0,0,0
5,0,0,0,0,0,0
...,...,...,...,...,...,...
362,0,0,0,0,0,0
364,0,0,0,0,0,0
368,0,0,0,0,0,0
369,0,0,0,0,0,0


In [ ]:
accuracy_score(1000, df[0], df[1])

0.7735849056603774

0.78284956233273


In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
print(accuracy_score(filtered_df[1], filtered_df[2]), accuracy_score(filtered_df[1], filtered_df[3]), accuracy_score(filtered_df[1], filtered_df[4]), accuracy_score(filtered_df[1], filtered_df[5]))

0.42160278745644597 0.9442508710801394 0.9442508710801394 0.9442508710801394


Multi Class & Multi Label

In [ ]:
import pandas as pd

In [ ]:
import transformers
import datasets
import torch
from pathlib import Path
from datasets import load_dataset
from transformers import (AutoTokenizer, AutoModelForSequenceClassification, 
                          TrainingArguments, Trainer)

In [ ]:
data=pd.read_csv('fulldata.csv')

In [ ]:
data

,text,label
0,Ghrelin was identified in the stomach as an en...,< < < <Sustaining proliferative signaling--Rec...
1,Leptin is thought to be involved in febrigenic...,< < < < < < < < <
2,PURPOSE The epidermal growth factor receptor (...,< < < < < < < <Sustaining proliferative signal...
3,Adoptive transfer of immunity against hepatiti...,< < < < < < < < < < < Avoiding immune destruc...
4,The secretion of immunosuppressive factors lik...,< < <Sustaining proliferative signaling--Growt...
...,...,...
1847,Tumour cells primarily utilize aerobic glycoly...,< < < < <Cellular energetics--Glycolysis/Warbu...
1848,Our previous study demonstrated that 5-aminole...,< < < <Cellular energetics--Glycolysis/Warburg...
1849,Ceramide is a sphingolipid metabolite that ind...,< < < < < <Resisting cell death--Necrosis<Cell...
1850,High-throughput screening of a small-molecule ...,< < < < <Cellular energetics--Glycolysis/Warbu...


In [ ]:
def find_sustain(text):
   if text.lower().find('sustaining proliferative signaling') == -1:
       return 0
   else:
       return 1
   
def find_evading(text):
    if text.lower().find('evading growth suppressors') == -1:
       return 0
    else:
       return 1
   
def find_resist(text):
   if text.lower().find('resisting cell death') == -1:
       return 0
   else:
       return 1
   
def find_enabling(text):
   if text.lower().find('enabling replicative immortality') == -1:
       return 0
   else:
       return 1
def find_Inducing(text):
   if text.lower().find('inducing angiogenesis') == -1:
       return 0
   else:
       return 1
   
def find_Activating(text):
   if text.lower().find('activating invasion') == -1:
       return 0
   else:
       return 1
   
def find_Genomic(text):
   if text.lower().find('genomic instability') == -1:
       return 0
   else:
       return 1
   
def find_Tumor(text):
   if text.lower().find('tumor promoting') == -1:
       return 0
   else:
       return 1
   
def find_Cellular(text):
   if text.lower().find('cellular energetics') == -1:
       return 0
   else:
       return 1
   
def find_Avoiding(text):
   if text.lower().find('avoiding immune destruction') == -1:
       return 0
   else:
       return 1

In [ ]:
data['Sustain proliferative signaling']=data['label'].apply(find_sustain)
data['Evading growth suppressors']=data['label'].apply(find_evading)
data['Resisting cell death']=data['label'].apply(find_resist)
data['Enabling replicative immortality']=data['label'].apply(find_enabling)
data['Inducing angiogenesis']=data['label'].apply(find_Inducing)
data['Activating invasion and metastasis']=data['label'].apply(find_Activating)
data['Genomic instability and mutation']=data['label'].apply(find_Genomic)
data['Tumor promoting inflammation']=data['label'].apply(find_Tumor)
data['Cellular energetics']=data['label'].apply(find_Cellular)
data['Avoiding immune destruction']=data['label'].apply(find_Avoiding)

In [ ]:
data.sum()

text                                  Ghrelin was identified in the stomach as an en...
label                                 < < < <Sustaining proliferative signaling--Rec...
Sustain proliferative signaling                                                     462
Evading growth suppressors                                                          240
Resisting cell death                                                                429
Enabling replicative immortality                                                    115
Inducing angiogenesis                                                               143
Activating invasion and metastasis                                                  291
Genomic instability and mutation                                                    333
Tumor promoting inflammation                                                        240
Cellular energetics                                                                 105
Avoiding immune destruction     

In [ ]:
multidataset=pd.DataFrame(data={'text':[],'Sustain proliferative signaling':[],'Evading growth suppressors':[],'Resisting cell death':[],'Enabling replicative immortality':[],'Inducing angiogenesis':[],'Activating invasion and metastasis':[],'Genomic instability and mutation':[],'Tumor promoting inflammation':[],'Cellular energetics':[],'Avoiding immune destruction':[]})

In [ ]:
multidataset=data.iloc[:,[0,2,3,4,5,6,7,8,9,10,11]]

In [ ]:
multidataset.to_feather('multidataset.csv')

In [ ]:
multidataset

,text,Sustain proliferative signaling,Evading growth suppressors,Resisting cell death,Enabling replicative immortality,Inducing angiogenesis,Activating invasion and metastasis,Genomic instability and mutation,Tumor promoting inflammation,Cellular energetics,Avoiding immune destruction
0,Ghrelin was identified in the stomach as an en...,1,0,0,0,0,0,0,0,0,0
1,Leptin is thought to be involved in febrigenic...,0,0,0,0,0,0,0,0,0,0
2,PURPOSE The epidermal growth factor receptor (...,1,1,1,0,0,0,0,0,0,0
3,Adoptive transfer of immunity against hepatiti...,0,0,0,0,0,0,0,0,0,1
4,The secretion of immunosuppressive factors lik...,1,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
1847,Tumour cells primarily utilize aerobic glycoly...,0,0,0,0,0,0,0,0,1,0
1848,Our previous study demonstrated that 5-aminole...,0,0,1,0,0,0,0,0,1,0
1849,Ceramide is a sphingolipid metabolite that ind...,0,0,1,0,0,0,0,0,1,0
1850,High-throughput screening of a small-molecule ...,0,0,0,0,0,0,0,0,1,0


In [ ]:
multidataset.to_csv('multidataset.csv',index=False)

In [ ]:
from sklearn.model_selection import train_test_split

multitrain, multitest = train_test_split(multidataset, test_size=0.2)

In [ ]:
multitest.reset_index().to_feather('multitest2.csv')

In [ ]:
multitrain.reset_index().to_feather('multitrain2.csv')

In [ ]:
from datasets import load_dataset
raw_datasets=load_dataset('csv',data_files={'train': 'multitrain.csv',
                                              'test':'multitest.csv'})

Using custom data configuration default-5efaad6f1e8e5747


100%|██████████| 2/2 [00:00<00:00, 2006.36it/s]


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xff in position 8: invalid start byte

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'Sustain proliferative signaling', 'Evading growth suppressors', 'Resisting cell death', 'Enabling replicative immortality', 'Inducing angiogenesis', 'Activating invasion and metastasis', 'Genomic instability and mutation', 'Tumor promoting inflammation', 'Cellular energetics', 'Avoiding immune destruction'],
        num_rows: 1481
    })
    test: Dataset({
        features: ['text', 'Sustain proliferative signaling', 'Evading growth suppressors', 'Resisting cell death', 'Enabling replicative immortality', 'Inducing angiogenesis', 'Activating invasion and metastasis', 'Genomic instability and mutation', 'Tumor promoting inflammation', 'Cellular energetics', 'Avoiding immune destruction'],
        num_rows: 371
    })
})

In [ ]:
raw_datasets['train'][0]

{'text': 'Genetic and epigenetic changes in the von Hippel-Lindau ( VHL ) tumour suppressor gene are common in sporadic conventional ( clear cell ) renal cell carcinoma ( ccRCC ) . The effects on VHL expression are unknown but increased understanding may be relevant clinically , either in terms of prognosis or in therapy selection . We have examined the expression of VHL mutant RNA in 84 ccRCC tumours previously screened for mutations in genomic DNA , 56 of which contained 52 unique mutations or polymorphisms . Based on the predicted change to the primary amino acid sequence , 24 of the mutations were missense , 11 resulted in frameshifts with premature truncation , 9 resulted in immediate truncation at the site of the mutation and 2 were frameshifts which extended the reading frame beyond the normal stop codon . Nine tumours had intronic variants , including substitution of invariant residues at splice sites , deletion of nucleotides spanning the exon-intron junction , an intronic var

Preprocessing

In [ ]:
# create labels column
cols = raw_datasets["train"].column_names


AttributeError: 'DatasetDict' object has no attribute 'to_feature'

In [ ]:
raw_datasets=raw_datasets.map(lambda x : {"labels": [x[c] for c in cols if c != "text"]})

1481ex [00:00, 17157.58ex/s]
371ex [00:00, 17726.11ex/s]


In [ ]:
raw_datasets['train'][0]

{'text': 'Genetic and epigenetic changes in the von Hippel-Lindau ( VHL ) tumour suppressor gene are common in sporadic conventional ( clear cell ) renal cell carcinoma ( ccRCC ) . The effects on VHL expression are unknown but increased understanding may be relevant clinically , either in terms of prognosis or in therapy selection . We have examined the expression of VHL mutant RNA in 84 ccRCC tumours previously screened for mutations in genomic DNA , 56 of which contained 52 unique mutations or polymorphisms . Based on the predicted change to the primary amino acid sequence , 24 of the mutations were missense , 11 resulted in frameshifts with premature truncation , 9 resulted in immediate truncation at the site of the mutation and 2 were frameshifts which extended the reading frame beyond the normal stop codon . Nine tumours had intronic variants , including substitution of invariant residues at splice sites , deletion of nucleotides spanning the exon-intron junction , an intronic var

In [ ]:
model_ckpt = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt, problem_type="multi_label_classification")

In [ ]:
def tokenize_and_encode(examples):
  return tokenizer(examples["text"], truncation=True)

In [ ]:
cols = raw_datasets["train"].column_names
cols.remove("labels")
ds_enc = raw_datasets.map(tokenize_and_encode, batched=True, remove_columns=cols)
ds_enc


100%|██████████| 1/1 [00:00<00:00, 17.92ba/s]


DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 1481
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 371
    })
})

In [ ]:
ds_enc

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 1481
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 371
    })
})

In [ ]:
ds_enc.set_format("torch")
ds_enc = (ds_enc
          .map(lambda x : {"float_labels": x["labels"].to(torch.float)}, remove_columns=["labels"])
          .rename_column("float_labels", "labels"))

1481ex [00:00, 2448.03ex/s]
371ex [00:00, 2885.59ex/s]


In [ ]:
num_labels=10
model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=num_labels, problem_type="multi_label_classification").to('cuda')

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'classifier.w

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate = 1e-5,
    #batch_size = 32,
    warmup_steps= 0,
    #max_seq_length = 128,
    num_train_epochs = 3.0 ,
    weight_decay=0.01,
    evaluation_strategy="epoch"
)

trainer = Trainer(model=model, args=training_args, train_dataset=ds_enc["train"], eval_dataset=ds_enc["test"], tokenizer=tokenizer)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
trainer.train()

C:\Users\RmmLeo10\anaconda3\envs\Leo\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1481
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 558
 33%|███▎      | 186/558 [00:15<00:28, 12.95it/s]***** Running Evaluation *****
  Num examples = 371
  Batch size = 8
                                                 
 34%|███▎      | 188/558 [00:16<01:31,  4.03it/s]

{'eval_loss': 0.16554871201515198, 'eval_runtime': 1.1103, 'eval_samples_per_second': 334.148, 'eval_steps_per_second': 42.331, 'epoch': 1.0}


 67%|██████▋   | 372/558 [00:32<00:13, 13.32it/s]***** Running Evaluation *****
  Num examples = 371
  Batch size = 8
                                                 
 67%|██████▋   | 374/558 [00:34<00:47,  3.90it/s]

{'eval_loss': 0.15805183351039886, 'eval_runtime': 1.1751, 'eval_samples_per_second': 315.726, 'eval_steps_per_second': 39.998, 'epoch': 2.0}


 90%|████████▉ | 500/558 [00:45<00:04, 11.75it/s]Saving model checkpoint to ./results\checkpoint-500
Configuration saved in ./results\checkpoint-500\config.json


{'loss': 0.13, 'learning_rate': 1.039426523297491e-06, 'epoch': 2.69}


Model weights saved in ./results\checkpoint-500\pytorch_model.bin
tokenizer config file saved in ./results\checkpoint-500\tokenizer_config.json
Special tokens file saved in ./results\checkpoint-500\special_tokens_map.json
100%|█████████▉| 557/558 [00:51<00:00, 11.88it/s]***** Running Evaluation *****
  Num examples = 371
  Batch size = 8
                                                 
100%|██████████| 558/558 [00:52<00:00, 11.88it/s]

Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 558/558 [00:52<00:00, 10.68it/s]

{'eval_loss': 0.1557828038930893, 'eval_runtime': 1.1741, 'eval_samples_per_second': 315.994, 'eval_steps_per_second': 40.032, 'epoch': 3.0}
{'train_runtime': 52.2422, 'train_samples_per_second': 85.046, 'train_steps_per_second': 10.681, 'train_loss': 0.12804148444992666, 'epoch': 3.0}


TrainOutput(global_step=558, training_loss=0.12804148444992666, metrics={'train_runtime': 52.2422, 'train_samples_per_second': 85.046, 'train_steps_per_second': 10.681, 'train_loss': 0.12804148444992666, 'epoch': 3.0})

In [ ]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds_enc["train"], 
    eval_dataset=ds_enc["test"],
    compute_metrics=compute_metrics,
)
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 371
  Batch size = 8


RuntimeError: stack expects each tensor to be equal size, but got [486] at entry 0 and [354] at entry 1

In [3]:
y_pred=[[1,1,1,0,1],[1,1,1,0,1],[1,1,1,0,1]]
y_label=[[1,0,0,1,1],[1,0,0,1,1],[1,0,0,1,1]]

In [4]:
from sklearn.metrics import accuracy_score
accuracy_score(y_label, y_pred)

0.0